In [1]:
import os
os.environ["HF_HOME"] = "/workspace/.cache/huggingface"

In [2]:
import pandas as pd
import numpy as np
import torch

model_path = "meta-llama/Llama-2-70b-chat-hf"

from transformers import AutoTokenizer,AutoModelForCausalLM, BitsAndBytesConfig

device = "cuda"

# quantization_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     llm_int8_threshold=6.0,
#     llm_int8_has_fp16_weight=False,
#     llm_int8_enable_fp32_cpu_offload=False
# )

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,  # or your specific Llama 70B variant
    quantization_config=quantization_config,
    device_map=device, 
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(model_path, device_map=device)

/workspace/projects/selfie/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/projects/selfie/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/workspace/projects/selfie/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/workspace/projects/selfie/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree

In [3]:
from selfie.interpret import InterpretationPrompt, interpret
interpretation_prompt = InterpretationPrompt(tokenizer, ("[INST]", 0, 0, 0, 0, 0, "[/INST] Sure, I will summarize the message:"))

/workspace/projects/selfie/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [10]:
num_layers = model.config.num_hidden_layers
print(f"Number of layers in model: {num_layers}")


Number of layers in model: 80


In [15]:
tokens = tokenizer.tokenize("[INST] What's highest mountain in the world? [/INST]")
for i, token in enumerate(tokens):
    print(i, token)

0 ▁[
1 INST
2 ]
3 ▁What
4 '
5 s
6 ▁highest
7 ▁mountain
8 ▁in
9 ▁the
10 ▁world
11 ?
12 ▁[
13 /
14 INST
15 ]


In [22]:
original_prompt = "[INST] What's highest mountain in the world? [/INST]"
tokens_to_interpret = [(40, 8), (60, 8)]
bs = 2
max_new_tokens = 20
k = 1

interpretation_df = interpret(original_prompt=original_prompt, tokens_to_interpret=tokens_to_interpret, model=model, interpretation_prompt=interpretation_prompt, bs=bs, max_new_tokens=max_new_tokens, k=k, tokenizer=tokenizer)

Interpreting '[INST] What's highest mountain in the world? [/INST]' with '[INST]_ _ _ _ _ [/INST] Sure, I will summarize the message:'


  0%|          | 0/1 [00:00<?, ?it/s]/workspace/projects/selfie/selfie/generate_wrappers.py:733: UserWarning: You are calling transformers.generation.generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
100%|██████████| 1/1 [00:09<00:00,  9.36s/it]


In [23]:
pd.DataFrame(interpretation_df)

,prompt,interpretation,layer,token,token_decoded,relevancy_score
0,[INST] What's highest mountain in the world? [...,\n\nThe message is: _ _ _ _\n\nPlease provide ...,40,8,mountain,"[0.1953, 0.1831, 0.04224, 0.8706, 0.413, 0.510..."
1,[INST] What's highest mountain in the world? [...,"\n\n""I don't know what to do. I'm feeling real...",60,8,mountain,"[0.0874, 0.253, 0.2974, 0.5884, 0.6553, 0.0014..."


In [24]:
print(pd.DataFrame(interpretation_df)['interpretation'][1])



"I don't know what to do. I'm feeling really down and hop
